### 라이브러리

In [19]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

In [20]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### API 키

In [21]:
# 🔐 환경변수 로드
load_dotenv()
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
OPENAI_API_KEY    = os.getenv("OPENAI_API_KEY")

True

### PDF 로드 / 임베딩

In [22]:
# 📄 PDF 로드 및 분할
loader = PyPDFLoader("KB 실버암 간편건강보험Plus.pdf")
pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size    = 300, 
                                               chunk_overlap = 30,
                                               )
docs = text_splitter.split_documents(pages)

# OpenAI 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",
                              openai_api_key = OPENAI_API_KEY,
                              )

# FAISS 벡터스토어 생성
vectorstore = FAISS.from_documents(docs, 
                                   embedding = embeddings,
                                   )
retriever = vectorstore.as_retriever()

In [ ]:
# Claude LLM
llm = ChatAnthropic(model="claude-opus-4-20250514",
                    temperature=0,
                    max_tokens=1024,
                    api_key=ANTHROPIC_API_KEY,
                    )

# RAG 프롬프트
prompt = hub.pull("rlm/rag-prompt")

# RAG 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# ✅ LLM 단독 체인 구성 (출력 깔끔하게)
llm_only_chain = llm | StrOutputParser()

# 질문
# question = "고지의무를 위반한 경우 보험사는 어떤 조치를 취할 수 있나요?"                                  # 민석 질문
# question = "특정암 ii란 뭐야?"                                                                          # 1번 질문
# question = "유사암진단비iii 를 청구하려고 하는데 갑상선암은 어떤 검사를 통해 진단받아야돼?"                # 2번 질문
# question = "통합암진단비ii 는 보장범위가 어떻게 돼?"                                                      # 3번 질문
question = "통합전이암진단비의 보장범위는 어떻게 돼?"                                                     # 4번 질문
# question = "보함계약한지 8개월이 지났는데 표적항암방사선 치료비를 보험가입금액의 100%를 받을 수 있어?"      # 5번 질문
# question = "신재진단암진단비ii 에서 수술이란 뭐야?"                                                       # 6번 질문    
# question = "보험금청구 했는데 언제 지급되니?"                                                             # 7번 질문

# 1. RAG 기반 응답
rag_answer = rag_chain.invoke(question)

# 2. 순수 LLM 응답 (문서 없이 바로 Claude에게 질문)
llm_only_answer = llm_only_chain.invoke(question)

# 출력
print("🧾 질문:", question)
print("\n💬 [LLM 단독 응답]:")
print(llm_only_answer)
print("\n📚 [RAG 기반 응답]:")
print(rag_answer)


C:\Users\KDT4\AppData\Roaming\Python\Python311\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


🧾 질문: 통합전이암진단비의 보장범위는 어떻게 돼?

💬 [LLM 단독 응답]:
통합전이암진단비의 일반적인 보장범위를 설명드리겠습니다.

## 주요 보장내용

**1. 전이암 진단 시**
- 원발암이 다른 장기로 전이된 경우
- 최초 1회에 한해 진단비 지급
- 조직검사 등으로 확진된 경우

**2. 보장 제외 사항**
- 원발암 자체는 보장하지 않음
- 재발암은 일반적으로 제외
- 전암성 병변(상피내암 등) 제외

**3. 주요 특징**
- 원발암 진단 후 전이암 발생 시 추가 보장
- 보험사별로 보장금액 차이 있음
- 면책기간/감액기간 적용 가능

## 확인 필요사항

보험상품마다 세부 약관이 다르므로:
- 가입하신 보험약관 확인
- 보장개시일 확인
- 진단비 지급조건 상세 확인

정확한 보장범위는 가입하신 보험사에 직접 문의하시거나 약관을 확인하시는 것이 가장 정확합니다.

📚 [RAG 기반 응답]:
제공된 문서에는 "통합전이암진단비"라는 특정 보장에 대한 직접적인 정보가 없습니다. 문서에는 폐전이암진단비, 간전이암·담낭전이암·기타담도전이암 및 췌장전이암진단비, 3대특정고액전이암진단비 등 개별 전이암 진단비들만 언급되어 있습니다. 통합전이암진단비의 구체적인 보장범위를 확인하려면 추가적인 정보가 필요합니다.


In [24]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate

# ✅ .env에서 API 키 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

# ✅ PDF 로드 및 문서 분할
loader = PyPDFLoader("KB 실버암 간편건강보험Plus.pdf")
pages = loader.load_and_split()

# 🔹 문맥 유지를 위한 적절한 청크 크기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=40
)
docs = text_splitter.split_documents(pages)

# ✅ OpenAI 임베딩 사용 (고품질 모델 추천)
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    openai_api_key=OPENAI_API_KEY
)

# ✅ FAISS 벡터스토어 생성
vectorstore = FAISS.from_documents(docs, embedding=embeddings)
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 8, "score_threshold": 0.6}
)

# ✅ Claude LLM 설정
llm = ChatAnthropic(
    model="claude-opus-4-20250514",
    temperature=0.2,
    max_tokens=2048,
    api_key=ANTHROPIC_API_KEY,
)

# ✅ 도메인 특화 프롬프트 정의
custom_prompt = PromptTemplate.from_template("""
당신은 복잡한 보험 약관을 쉽게 풀어 설명해주는 스마트 상담사입니다.
아래는 KB 실버암 간편건강보험Plus 약관의 일부입니다.

---------------------
{context}
---------------------

사용자의 질문에 대해 반드시 문서 내용에 기반해 답변하세요.  
질문자가 이해하기 쉽도록 **간결하고 쉬운 표현**을 사용하세요.

❗주의사항:
- 문서에 없는 내용은 "해당 내용은 약관에 명시되어 있지 않습니다"라고 답하세요.
- 약관 내용이 어렵거나 법률 용어가 있다면 쉬운 용어로 풀어서 설명하세요.
- 숫자(금액/기간/횟수)는 명확하게 표현하세요. 예: “90일”, “1회만 지급” 등

질문: {question}
답변:
""")


# ✅ RAG 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | custom_prompt
    | llm
    | StrOutputParser()
)

# ✅ LLM 단독 체인 (문서 없이 바로 질문)
llm_only_chain = llm | StrOutputParser()

# ✅ 테스트 질문
# question = "통합전이암진단비의 보장범위는 어떻게 돼?"
# question = "통합전이암진단비 갱신계약은 어떤 항목을 보장하나요?"
question = "통합전이암진단비 관련된 정보 니가 알고있는 만큼 다 알려줘"

# ✅ 1. RAG 기반 응답
rag_answer = rag_chain.invoke(question)

# ✅ 2. LLM 단독 응답
llm_only_answer = llm_only_chain.invoke(question)

# ✅ 결과 출력
print("🧾 질문:", question)
print("\n💬 [LLM 단독 응답]:")
print(llm_only_answer)

print("\n📚 [RAG 기반 응답]:")
print(rag_answer)


True

No relevant docs were retrieved using the relevance score threshold 0.6


🧾 질문: 통합전이암진단비 관련된 정보 니가 알고있는 만큼 다 알려줘

💬 [LLM 단독 응답]:
통합전이암진단비에 대해 제가 알고 있는 정보를 상세히 설명드리겠습니다.

## 통합전이암진단비란?

암보험의 특약 중 하나로, 원발암(최초 발생한 암)이 다른 장기나 조직으로 전이되었을 때 지급되는 진단비입니다.

## 주요 특징

### 1. 보장 내용
- **전이암 진단 시** 정액의 진단비 지급
- 원발암과는 별도로 보장
- 일반적으로 1회만 지급 (상품에 따라 다름)

### 2. 전이암의 정의
- 원발암이 혈관이나 림프관을 통해 다른 장기로 퍼진 암
- 원격전이암과 국소전이암 모두 포함
- 재발암과는 구별됨

### 3. 보장 조건
- **원발암 진단 후** 일정 기간 경과 필요 (보통 90일~1년)
- 조직검사를 통한 확진 필요
- 원발암 진단일로부터 일정 기간 내 전이 진단 시 보장

## 가입 시 주의사항

### 1. 면책기간
- 보통 90일의 면책기간 존재
- 원발암 진단 후 추가 면책기간이 있을 수 있음

### 2. 감액기간
- 가입 후 1~2년간 보험금의 50%만 지급하는 경우가 많음

### 3. 보장 제외 사항
- 원발암 진단 전 발견된 전이암
- 특정 암종의 전이 제외 가능
- 갑상선암, 기타피부암 등의 전이는 제외될 수 있음

## 보험금 지급 기준

### 1. 진단 확정
- 병리학적 검사 필수
- 영상검사만으로는 불충분
- 전문의의 진단서 필요

### 2. 지급 금액
- 가입금액의 100% (감액기간 제외)
- 상품에 따라 500만원~5,000만원 수준

### 3. 중복 지급
- 여러 장기로 전이되어도 1회만 지급이 일반적
- 일부 상품은 장기별 지급 가능

## 상품별 차이점

### 1. 보장 범위
- **광범위 보장**: 모든 전이암 보장
- **제한적 보장**: 특정 장기 전이만 보장

### 2. 지급 횟수
- **1회 한정**: 최초 1회만 지급
- **다회 지급**: 장기별 또는 일정 기간 후 재지급

### 3.